## Lendo dados JSON

In [ ]:
import pandas as pd

In [ ]:
path = 'dados/aula4/nyc_death_causes.json'
nyc_death_causes = pd.read_json(path)

In [ ]:
nyc_death_causes

## Carregando dados de uma API

Vamos usar a Yelp API como exemplo 
https://docs.developer.yelp.com/docs/fusion-intro

In [ ]:
import requests

In [ ]:
api_url = "https://api.yelp.com/v3/businesses/search"

params = {"term":"bookstore",
          "location":"San Francisco"}
api_key =  'E75fkz2z6r266cXTaRvTnMSmAuvEq9pqhtsX8oqRxRhP3mJ1wExtiGixZ8wWjNWnX_PgBDtdNoXpm57gWKIxZ0VOZ0emVSC1aOkZFLohRBsiYaxXzHa44-bPZoobZHYx'
headers = {"Authorization":"Bearer {}".format(api_key)}
response = requests.get(api_url, params = params, headers=headers)

In [ ]:
response

In [ ]:
data = response.json()
data

In [ ]:
bookstores = pd.DataFrame(data['businesses'])
bookstores.head()

## Trabalhando com JSONs aninhados

In [ ]:
bookstores[['categories', 'coordinates', 'location']].head()

In [ ]:
bookstores = pd.json_normalize(data['businesses'], sep ="_")

In [ ]:
type(bookstores['transactions'][0])
## snake case coordinates_latitude
## camel case CoordinatesLatitude

### JSONs profundamente aninhados

In [ ]:
df = pd.json_normalize(data['businesses'], 
                 record_path='categories',
                  sep ="_",
                  meta = ['name', 'alias' , 'rating', ['coordinates', 'latitude'], ['coordinates', 'longitude']],
                  meta_prefix = "business_"
                  )
df.head()

### Busca paginada

Por padrão a yelp api retorna os 20 primeiros items

Pegando as primeiras 20 lojas

In [ ]:
params = {"term": "bookstore",
        "location":"San Francisco"} 
first_results = requests.get(api_url, headers=headers, params=params).json() 
first_20_bookstores = pd.json_normalize(first_results["businesses"], sep="_") 
first_20_bookstores.shape

Pegando as próximas 20 lojas

In [ ]:
params['offset'] = 20
next_results = requests.get(api_url, headers=headers, params=params).json() 
next_20_bookstores = pd.json_normalize(next_results["businesses"], sep="_") 

In [ ]:
next_20_bookstores.tail()

Pegando as primeiras 40 lojas

In [ ]:
params ={'term': 'bookstore', 
         'location': 'San Francisco', 
         'offset': 40, 
         'limit': 40}

In [ ]:
params['limit'] = 40
next_results = requests.get(api_url, headers=headers, params=params).json() 
next_40_bookstores = pd.json_normalize(next_results["businesses"], sep="_") 

In [ ]:
next_40_bookstores.shape

### Juntando Dataframes

In [ ]:
pd.concat([first_20_bookstores, next_20_bookstores]).reset_index(drop = True).head(1)

### Paginação completa